In [1]:
# set environment and import packages
import os
from pandas import read_csv
import pandas as pd
import geopandas as gps
import matplotlib.pyplot as plt
import seaborn as sns
from os import listdir
import constants as c
import warnings
warnings.filterwarnings("ignore")

os.chdir('/Users/xiaodanxu/Documents/SynthFirm.nosync/mode_choice')



In [2]:
sctg_group_lookup = read_csv(c.param_dir + c.sctg_group_lookup_file, sep = ',')
mesozone_lookup = read_csv(c.param_dir + c.mesozone_id_lookup_file, sep = ',')

truck_mode = ['For-hire Truck', 'Private Truck']
# max_ton_lookup = {'sctg1': 218.327, 'sctg2': 51.13091, 'sctg3': 94.56035, 'sctg4': 4.4861, 'sctg5': 51.64694}

In [3]:
combined_modeled_OD = None
mode_choide_by_commodity = None
combined_truck_output = None
for sctg in c.list_of_sctg_group:
    print(sctg)
    b2b_dir = c.output_dir + sctg
    list_of_b2b_files = listdir(b2b_dir)
    iterator = 0
#     cut_off_point = max_ton_lookup[sctg] # shipment capacity for this shipment
    for file in list_of_b2b_files:
        if file == '.DS_Store':
            continue
        if iterator%10 == 0:
            print(iterator)
        modeled_OD_by_sctg = read_csv(b2b_dir + '/' + file, sep = ',')
#         print(modeled_OD_by_sctg.columns)
        list_of_var = ['BuyerID', 'BuyerZone', 'BuyerNAICS', 'SellerID',
           'SellerZone', 'SellerNAICS', 'TruckLoad', 'SCTG_Group', 
           'shipment_id', 'orig_FAFID', 'dest_FAFID', 
           'mode_choice', 'probability', 'Distance', 'Travel_time']
#         print(modeled_OD_by_sctg.head(5))
        truck_output = modeled_OD_by_sctg.loc[modeled_OD_by_sctg['mode_choice'].isin(truck_mode), list_of_var]
        int_var = ['BuyerID', 'BuyerZone', 'SellerID',
           'SellerZone', 'SCTG_Group', 
           'shipment_id', 'orig_FAFID', 'dest_FAFID']
        truck_output.loc[:, int_var] = truck_output.loc[:, int_var].astype(int)
        truck_output.to_csv(c.output_dir + sctg + '_truck/truck_only_OD_' + sctg + '_' + 
                            str(iterator) + '.csv', index = False)

        
        ## compute national shipment count and tonmile
        modeled_OD_by_sctg['ShipmentLoad'] = modeled_OD_by_sctg['TruckLoad'] / 1000 # convert to thousand tons
        
        #print(cut_off_point)
#         modeled_OD_by_sctg.loc[modeled_OD_by_sctg['ShipmentLoad'] > cut_off_point, 'ShipmentLoad'] = cut_off_point
        modeled_OD_by_sctg['tmiles'] = modeled_OD_by_sctg['ShipmentLoad'] * 1000 * modeled_OD_by_sctg['Distance']
        modeled_OD_by_sctg = pd.merge(modeled_OD_by_sctg, mesozone_lookup, 
                                      left_on = ['SellerZone', 'orig_FAFID'], 
                                    right_on = ['MESOZONE', 'FAFID'], how = 'left')
        modeled_OD_by_sctg = modeled_OD_by_sctg.rename(columns={"GEOID": "orig_GEOID",
                                                                "CBPZONE": "orig_CBPZONE", 
                                                                "MESOZONE":"orig_MESOZONE", 
                                                                "FAFNAME":"orig_FAFNAME"})
        modeled_OD_by_sctg = pd.merge(modeled_OD_by_sctg, mesozone_lookup, 
                                      left_on = ['BuyerZone', 'dest_FAFID'], 
                                    right_on = ['MESOZONE', 'FAFID'], how = 'left')
        modeled_OD_by_sctg = modeled_OD_by_sctg.rename(columns={"GEOID": "dest_GEOID", 
                                                                "CBPZONE": "dest_CBPZONE", 
                                                                "MESOZONE":"dest_MESOZONE", 
                                                               "FAFNAME":"dest_FAFNAME"})    
        agg_OD_by_sctg = modeled_OD_by_sctg.groupby(["orig_FAFID", "orig_FAFNAME", "dest_FAFID", "dest_FAFNAME", "SCTG_Group", 'mode_choice'])[['tmiles', 'ShipmentLoad']].sum()        
        agg_OD_by_sctg = agg_OD_by_sctg.reset_index()
        agg_count_by_sctg = modeled_OD_by_sctg.groupby(["orig_FAFID", "orig_FAFNAME", "dest_FAFID", "dest_FAFNAME", "SCTG_Group", 'mode_choice'])[['shipment_id']].count() 
        agg_count_by_sctg = agg_count_by_sctg.reset_index()
        agg_OD_by_sctg = pd.merge(agg_OD_by_sctg, agg_count_by_sctg, 
                                  on = ["orig_FAFID", "orig_FAFNAME", "dest_FAFID", "dest_FAFNAME", "SCTG_Group", 'mode_choice'],
                                  how = 'left')
        agg_OD_by_sctg = agg_OD_by_sctg.rename(columns={"shipment_id": "count"})
        agg_OD_by_sctg.loc[:, 'SCTG_Name'] = c.sctg_def[sctg]
        agg_OD_by_sctg.loc[:, 'chunk_id'] = iterator
        combined_modeled_OD = pd.concat([combined_modeled_OD, agg_OD_by_sctg], sort = False)
        iterator += 1 
#         break        
#     break
#     combined_truck_output.to_csv(c.input_dir + 'truck_only_OD_' + sctg + '.csv', index = False)
# combined_modeled_OD = pd.merge(combined_modeled_OD, sctg_group_definition, on = ['SCTG_Group'], how = 'left')
combined_modeled_OD.head(10)    

sctg1
0
sctg2
0
10
sctg3
0
sctg4
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
sctg5
0


,orig_FAFID,orig_FAFNAME,dest_FAFID,dest_FAFNAME,SCTG_Group,mode_choice,tmiles,ShipmentLoad,count,SCTG_Name,chunk_id
0,41,Phoenix,64,San Francisco,1,For-hire Truck,4.529535e+09,4460.442945,132461,bulk,0
1,41,Phoenix,64,San Francisco,1,Rail/IMX,1.980990e+09,2158.581607,64103,bulk,0
2,61,Los Angeles,69,Rest of CA,1,For-hire Truck,2.468399e+09,3989.654672,118482,bulk,0
3,61,Los Angeles,69,Rest of CA,1,Rail/IMX,5.585304e+08,846.103615,25127,bulk,0
4,64,San Francisco,64,San Francisco,1,For-hire Truck,0.000000e+00,591.704811,17545,bulk,0
5,64,San Francisco,64,San Francisco,1,Private Truck,0.000000e+00,563.414464,16706,bulk,0
6,64,San Francisco,64,San Francisco,1,Rail/IMX,0.000000e+00,29.340000,870,bulk,0
7,64,San Francisco,69,Rest of CA,1,For-hire Truck,7.156719e+07,450.504801,13380,bulk,0
8,64,San Francisco,69,Rest of CA,1,Private Truck,2.309618e+07,145.387028,4318,bulk,0
9,64,San Francisco,69,Rest of CA,1,Rail/IMX,8.905451e+06,38.484374,1143,bulk,0


In [4]:
combined_modeled_OD_agg = combined_modeled_OD.groupby(["orig_FAFID", "orig_FAFNAME", "dest_FAFID", \
                                                       "dest_FAFNAME", "SCTG_Group", 'SCTG_Name',
                                                       'mode_choice'])[['tmiles', 'ShipmentLoad', 'count']].sum()
combined_modeled_OD_agg = combined_modeled_OD_agg.reset_index()
combined_modeled_OD_agg.head(5)
# combined_modeled_OD_agg.loc[:, 'in_study_area'] = 0
# buffer = combined_modeled_OD_agg.loc[:, 'orig_FAFID'].isin(c.bay_area_region_code) | \
#         combined_modeled_OD_agg.loc[:, 'dest_FAFID'].isin(c.bay_area_region_code)
# combined_modeled_OD_agg.loc[buffer, 'in_study_area'] = 1

combined_modeled_OD_agg.loc[:, 'outbound'] = 0
combined_modeled_OD_agg.loc[combined_modeled_OD_agg.loc[:, 'orig_FAFID'].isin(c.region_code), 'outbound'] = 1

combined_modeled_OD_agg.loc[:, 'inbound'] = 0
combined_modeled_OD_agg.loc[combined_modeled_OD_agg.loc[:, 'dest_FAFID'].isin(c.region_code), 'inbound'] = 1

combined_modeled_OD_agg.loc[:, 'orig_FAFID'] = combined_modeled_OD_agg.loc[:, 'orig_FAFID'].astype(int)
combined_modeled_OD_agg.loc[:, 'dest_FAFID'] = combined_modeled_OD_agg.loc[:, 'dest_FAFID'].astype(int)
combined_modeled_OD_agg.loc[:, 'SCTG_Group'] = combined_modeled_OD_agg.loc[:, 'SCTG_Group'].astype(int)
combined_modeled_OD_agg.loc[:, 'Distance'] = combined_modeled_OD_agg.loc[:, 'tmiles'] / 1000 / combined_modeled_OD_agg.loc[:, 'ShipmentLoad']
combined_modeled_OD_agg.head(10)

,orig_FAFID,orig_FAFNAME,dest_FAFID,dest_FAFNAME,SCTG_Group,SCTG_Name,mode_choice,tmiles,ShipmentLoad,count,outbound,inbound,Distance
0,11,Birmingham,62,Sacramento,4,mfr_goods,Air,3.142313e+03,0.001620,30,0,1,1940.092929
1,11,Birmingham,62,Sacramento,4,mfr_goods,For-hire Truck,4.771059e+03,0.001728,32,0,1,2761.590000
2,11,Birmingham,62,Sacramento,4,mfr_goods,Parcel,3.163261e+04,0.016305,302,0,1,1940.092929
3,11,Birmingham,62,Sacramento,4,mfr_goods,Rail/IMX,1.389176e+02,0.000054,1,0,1,2573.070582
4,11,Birmingham,64,San Francisco,1,bulk,Rail/IMX,1.019738e+06,0.396585,1,0,1,2571.300823
5,11,Birmingham,65,Fresno,4,mfr_goods,Air,6.747436e+05,0.361221,1560,0,1,1867.950344
6,11,Birmingham,65,Fresno,4,mfr_goods,For-hire Truck,7.569925e+06,3.039386,13125,0,1,2490.610000
7,11,Birmingham,65,Fresno,4,mfr_goods,Rail/IMX,6.587909e+04,0.025256,109,0,1,2608.500582
8,11,Birmingham,69,Rest of CA,1,bulk,For-hire Truck,2.019337e+06,0.745754,29,0,1,2707.780000
9,11,Birmingham,69,Rest of CA,1,bulk,Rail/IMX,2.097460e+07,8.254724,321,0,1,2540.920582


In [5]:
combined_modeled_OD_agg.to_csv(c.output_dir + 'processed_b2b_flow_summary.csv', sep = ',')